<a href="https://colab.research.google.com/github/soprunveronika/DSL/blob/main/Job2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Здесь реализована функция удаления ненужных нетерминальных символов. Алгоритм состоит из того, что мы итеративно заполняем наш набор полезных нетерминальных символов, что может привести нас к последовательности, где все символы являются конечными. На каждой итерации мы ищем нетерминалы, у которых есть правило соответствия последовательности, где все символы являются терминалами, или уже известные полезные нетерминалы. Мы останавливаем наши итерации после того, как не можем найти новые полезные нетерминалы после того, как перебрали весь набор нетерминалов.

Структура нашей контекстно-свободной грамматики такова, что:
{'toks': set(token), 'vars': dict(var: definition), 'hvar': var}
token : (class, value)    # это просто терминальный символ 
class : int
value : str
var : str                 # имя нетерминального 
definition : list(rule)
rule : list(var | token)  # правая часть правила сопоставления

In [ ]:
def remove_all_usefull_non_terminals(grammar):

  usefull_non_terminals = set()   # это пустой набор 
  tokens = grammar['toks']
  variables = grammar['vars']
  start_non_terminal = grammar['hvar']
  found_new_usefull_non_terminal = True # установка true только для входа в цикл

  while found_new_usefull_non_terminal:
    found_new_usefull_non_terminal = False
    for non_terminal, definition in variables.items():
      if non_terminal not in usefull_non_terminals:
        for rule in definition:
          # если для всех символов из правила является терминальным или полезным нетерминальным 
          if all(map(lambda symbol: symbol in tokens or symbol in usefull_non_terminals, rule)): 
            found_new_usefull_non_terminal = True
            usefull_non_terminals.add(non_terminal)
            break   # мы определили, что текущий символ полезен, поэтому больше не повторяем

  # новый набор нетерминалов. pair[0] является ключом в паре "ключ-значение" в словаре 
  new_variables = dict(filter(lambda pair: pair[0] in usefull_non_terminals, variables.items()))

  # очищение всех правил, которые являються бесполезные нетерминалы
  for non_terminal in new_variables.keys():
    old_definition = new_variables[non_terminal]
    new_variables[non_terminal] = list(
        filter(lambda rule: all(
            map(lambda symbol: symbol in tokens or symbol in usefull_non_terminals, rule)
        ), old_definition)
    )
  
  # удаление стартового нетерминала, если его больше нет в новом наборе
  if start_non_terminal not in usefull_non_terminals:
    start_non_terminal = None

  return {'toks': tokens, 'vars': new_variables, 'hvar': start_non_terminal}

Далее следует реализация функции, которая удаляет все истекающие нетерминалы. Алгоритм похожий. Мы итеративно заполняем набор истекающих нетерминалов, пока он не перестанет расти после прохождения всех правил сопоставления. Добавляем нетерминал в набор, если его еще нет и есть правило, в правой части которого все символы находятся в наборе истекающих нетерминалов. 

In [ ]:
def find_all_expiring_non_terminals(grammar):

  expiring_non_terminals = set()   # это пустой набор
  tokens = grammar['toks']
  variables = grammar['vars']
  start_non_terminal = grammar['hvar']
  found_new_expiring_non_terminal = True # установка true только для входа в цикл 

  while found_new_expiring_non_terminal:
    found_new_expiring_non_terminal = False
    for non_terminal, definition in variables.items():
      if non_terminal not in expiring_non_terminals:
        for rule in definition:
          # если для всех символов из правила истекает срок действия
          if all(map(lambda symbol: symbol in expiring_non_terminals, rule)): 
            found_new_expiring_non_terminal = True
            expiring_non_terminals.add(non_terminal)
            break   # мы определили, что текущий символ истекает, поэтому больше не повторяем

  return expiring_non_terminals